In [6406]:
# imports
import tkinter as tk
from tkinter import ttk
import colorsys
from PIL import ImageGrab
import random
import string
from time import sleep
from numpy.random import choice
import numpy as np

In [6407]:
def norm(x):
    return [i/sum(x) for i in x]

In [6408]:
# ----------------------------------------
# constants
MIN_WIDTH = 80
MAX_WIDTH = 200

MIN_HEIGHT = 30
MAX_HEIGHT = 100

MERGE_RATE_EDGE = 0.1
MERGE_RATE = 0.05

WIDGET_LIST = ["empty", "button", "label", "entry"]
WIDGET_PROB = norm([100,0.1,0.1,0.1])
WIDGET_EDGE_PROB = norm([100,10,10,5])
WIDGET_NEIGHBOUR_PROB = norm([100,10,10,5])

# WIDGET_LIST = ["empty", "label", "button"]
# WIDGET_PROB = [0.0, 0.5, 0.5]

In [6409]:
def random_string(min_len, max_len):
    length = random.randint(min_len, max_len)
    letters = string.ascii_letters  # a-z + A-Z
    return ''.join(random.choice(letters) for _ in range(length))

In [6410]:
# color palette generator

# def random_color():
#     return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# def generate_color_palette(n, palette_size=5, repeat_chance=0.3):
#     # Generate a base palette of distinct colors
#     palette = [random_color() for _ in range(palette_size)]
    
#     colors = []
#     for _ in range(n):
#         if colors and random.random() < repeat_chance:
#             # Repeat an existing color
#             colors.append(random.choice(colors))
#         else:
#             # Pick a new color from the palette (some might repeat too)
#             colors.append(random.choice(palette))
    
#     return colors

def hsl_to_hex(h, s, l):
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))

GUI_BASE_HUES = [
    0.55,  # muted blue
    0.33,  # green
    0.08,  # orange
    0.75,  # purple
    0.0,   # red
    0.16,  # brown/tan
    0.6,   # teal
    0.9    # pink/magenta
]

def generate_color_palette(n, palette_size=5, repeat_chance=0.3, white_chance=0.2, grey_chance=0.15):
    """
    Generate a harmonious GUI-friendly color palette with a chance for white.
    
    Parameters:
        n (int): Number of colors to generate.
        palette_size (int): Number of base colors in the palette.
        repeat_chance (float): Probability of repeating a color.
        white_chance (float): Probability of a palette color being white.
    """
    palette = []
    for i in range(palette_size):
        if random.random() < white_chance:
            palette.append("#ffffff")  # Add white directly
        elif random.random() < grey_chance:
            palette.append("#b2b2b2")  # Add grey directly
        else:
            h = random.choice(GUI_BASE_HUES)
            s = random.uniform(0.2, 0.45)
            l = random.uniform(0.5, 0.85)
            palette.append(hsl_to_hex(h, s, l))
    
    # Now build the final list with some repeats
    colors = []
    for _ in range(n):
        if colors and random.random() < repeat_chance:
            colors.append(random.choice(colors))
        else:
            colors.append(random.choice(palette))
    
    return colors

n = 15
color_list = generate_color_palette(n, palette_size=6, repeat_chance=0.2)
print(color_list)

['#b06a6a', '#b06a6a', '#b06a6a', '#cf9090', '#ffffff', '#b06a6a', '#ffffff', '#b06a6a', '#d99f9f', '#c4c17e', '#ffffff', '#ffffff', '#b06a6a', '#ffffff', '#ffffff']


In [ ]:
# ----------------------------------------
# the widget grid
# ----------------------------------------

root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.destroy()
root.mainloop()

# generating rows and columns

# columns
row_coords = [0]
column_coords = [0]

while 1:
    new_x = random.randint(MIN_WIDTH, MAX_WIDTH)

    if (row_coords[-1] + new_x) > screen_width:
        # row_coords.append( screen_width) # uncommented: can result in small cells
        row_coords[-1] = screen_width
        break
    else:
        row_coords.append( row_coords[-1] + new_x)

while 1:
    new_y = random.randint(MIN_HEIGHT, MAX_HEIGHT)

    if (column_coords[-1] + new_y) > screen_height:
        # column_coords.append( screen_height) # uncommented: can result in small cells
        column_coords[-1] = screen_height
        break
    else:
        column_coords.append( column_coords[-1] + new_y)

print(row_coords)
print(column_coords)

[0, 149, 276, 450, 601, 714, 865, 950, 1066, 1163, 1274, 1446, 1536]
[0, 83, 159, 246, 300, 393, 465, 499, 595, 679, 763, 864]


In [6412]:
class cell:

    def __init__(self, lx, rx, uy, dy, ):

        self.lx = lx
        self.rx = rx
        self.uy = uy
        self.dy = dy

        self.edge = (lx == 0) or (uy == 0) or\
                    (rx == screen_width) or (dy == screen_height)
        
        self.active = True
        self.merged = False
        self.widget = ""
        self.neighbours = []

In [6413]:
# creating cells within a grid

grid = []
cell_list = []

for i,r in enumerate(row_coords[:-1]):
    grid.append([])
    for j,c in enumerate(column_coords[:-1]):

        new_cell = cell(r, row_coords[i+1],
                        c,column_coords[j+1])
        
        grid[-1].append(new_cell)
        cell_list.append(new_cell)

In [6414]:
# finding neighbours and merging

max_row = len(grid) - 1
max_col = len(grid[0]) - 1


def add_neighbours(c):
    if row > 0:
        c.neighbours.append( grid[row-1][col])
    if row < max_row:
        c.neighbours.append( grid[row+1][col])
    if col > 0:
        c.neighbours.append( grid[row][col-1])
    if col < max_col:
        c.neighbours.append( grid[row][col+1])
    return

def merge_cells(c):
    for n in c.neighbours:
        if (not c.merged) and (not n.merged):
            merge_chance = random.random()

            if merge_chance < MERGE_RATE:
                n.active = False
                n.merged = True
                c.merged = True
                c.lx = min( c.lx, n.lx)
                c.uy = min( c.uy, n.uy)
                c.rx = max( c.rx, n.rx)
                c.dy = max( c.dy, n.dy)
    return

def assign_widget(c):
    if len([n.widget for n in c.neighbours if n.widget not in ["","empty"]]):
        widget = choice( WIDGET_LIST, 1, p=WIDGET_NEIGHBOUR_PROB)
    elif c.edge:
        widget = choice( WIDGET_LIST, 1, p=WIDGET_EDGE_PROB)
    else:
        widget = choice( WIDGET_LIST, 1, p=WIDGET_PROB)
        
    c.widget = widget[0]
    return

for row,_ in enumerate(grid):
    for col,_ in enumerate(grid[row]):
        c = grid[row][col]

        if c.active:
            add_neighbours(c)
            merge_cells(c)
            assign_widget(c)

In [6415]:
cell_list = []

for row,_ in enumerate(grid):
    for col,_ in enumerate(grid[row]):
        c = grid[row][col]

        if c.active:
            cell_list.append(c)

In [6416]:
# sleep(5)

In [6417]:
class MainApplication(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, background=color_list[0], *args, **kwargs)
        self.parent = parent

        for c in cell_list:
            if c.widget == "button":
                ttk.Button( self, text="button_"+random_string(5, 10)).place(x=c.lx,y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)
            if c.widget == "label":
                ttk.Label( self, text="label_"+random_string(5, 10)).place(x=c.lx, y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)
            if c.widget == "entry":
                ttk.Entry( self).place(x=c.lx, y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)

def take_screenshot_and_quit(root):
    root.update()
    screenshot = ImageGrab.grab()
    screenshot.save("screenshots/screenshot.png")
    screenshot.close()
    root.destroy()

if __name__ == "__main__":
    root = tk.Tk()

    # random style
    style = ttk.Style(root)
    themes = style.theme_names() # available themes
    style.theme_use(random.choice(themes))

    # Configure styles for each widget type
    style.configure("TButton", background=color_list[1], foreground="black")
    style.configure("TLabel", background=color_list[2], foreground="black")
    style.configure("TEntry", fieldbackground=color_list[3], foreground="black")

    root.geometry(f"{screen_width}x{screen_height}")
    root.attributes("-fullscreen", True)
    root.attributes("-topmost", True)
    MainApplication(root).pack(side="top", fill="both", expand=True)

    root.after(500, lambda: take_screenshot_and_quit(root))

    root.mainloop()